# Constants

In [1]:
import os, json
from IPython.display import Markdown, display
from dotenv import load_dotenv # requires python-dotenv
from PIL import Image # requires pip install pillow
from datetime import datetime
from common.agents_helper_functions_NEW import *
import importlib.metadata
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

if not load_dotenv("./../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")


project_endpoint = os.environ["AZURE_AIFSTDEASTUS_PROJECT_ENDPOINT"]
deployment_name =  os.environ["MODEL_DEPLOYMENT_NAME"]
api_version = "2025-05-15-preview" # os.environ["OPENAI_API_VERSION"] # at least "2025-03-01-preview"

print(f'Project Endpoint: <{project_endpoint}>')
print(f"azure-ai-projects library installed version: {importlib.metadata.version("azure-ai-projects")}")
print(f"azure-ai-agents library installed version: {importlib.metadata.version("azure-ai-agents")}")

Environment variables have been loaded ;-)
Project Endpoint: <https://basicfdrymlbe.services.ai.azure.com/api/projects/project>
azure-ai-projects library installed version: 1.0.0b12
azure-ai-agents library installed version: 1.1.0b2


# [Create AI Foundry Agent Client](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python-preview)
**Note**: I could create the `project` client rather than the `agent` client, however this is easier to read.<br/>
Please consider that `project_client.agens == agents_client`

In [2]:
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential



from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

project_client

# [MCP Agent](https://devblogs.microsoft.com/foundry/announcing-model-context-protocol-support-preview-in-azure-ai-foundry-agent-service/)
- [Connect to Model Context Protocol (MCP) Servers (Preview)](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol)
- [How to use the Model Context Protocol (MCP) tool WITH http](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol-samples)
- [AI Foundry Basic Agent Setup](https://github.com/azure-ai-foundry/foundry-samples/tree/main/samples/microsoft/infrastructure-setup/40-basic-agent-setup)
- [Announcing Model Context Protocol Support (preview) in Azure AI Foundry Agent Service](https://devblogs.microsoft.com/foundry/announcing-model-context-protocol-support-preview-in-azure-ai-foundry-agent-service/)
- Region: WestUS

aif02westusgrp
basicfdrydl5j
aif02basicproject01

In [3]:
# MCP Agent Instructions
agent_name = "MCP_learn_agent"

agent_instructions = "You are a customer support chatbot. Use the tools provided and your knowledge base to best respond to customer " \
"queries about Microsoft Learn portal."

# Question
question = "Quali sono i modelli Azure OpenAI in scadenza e con cosa posso sostituirli? " \
"Fornisci dettagli per ogni modello e versione, evidenziando le differenze tra le versioni. " \
"Includi anche le date di scadenza e le versioni alternative consigliate."

In [16]:
# MCP Tool definition

mcp_tool_definition = {
    "type": "mcp", #il tool è di tipo MCP
    "server_label": "microsoft_docs_search", #il nome univoco per MCP Server di MSLearn è "microsoft_docs_search"
    "server_url": "https://learn.microsoft.com/api/mcp", #l'URL del server MCP di MSLearn
    # "require_approval": "never" 
}

In [17]:
# Agent creation

assistant_id = "" # ex: asst_j1qWBdGsjbK4hHcO0M0n3M5p. If provided, it will be loaded rather than created

if assistant_id != "":
    agent = project_client.agents.get_agent(assistant_id=assistant_id)
else:
    agent = project_client.agents.create_agent(
        model=deployment_name,
        name=agent_name,
        instructions=agent_instructions,
        tools=[mcp_tool_definition], # leave this line commented to create a naked agent ;-)
        tool_resources=None,
        headers={"x-ms-enable-preview": "true"}
    )

agent.as_dict()

HttpResponseError: (unknown_parameter) Unknown parameter: 'tools[0].approval'.
Code: unknown_parameter
Message: Unknown parameter: 'tools[0].approval'.

# Create the thread and attach a new message to it

In [8]:
# Create a thread
thread = project_client.agents.threads.create()
print(f"Created thread: {thread}\n")

Created thread: {'id': 'thread_M7JWwNMqn9JB5Eer2TVjI6P3', 'object': 'thread', 'created_at': 1751467171, 'metadata': {}, 'tool_resources': {}}



In [9]:
# Add a user message to the thread
from azure.ai.agents.models import MessageRole

message = project_client.agents.messages.create(
    thread_id=thread.id, 
    role=MessageRole.USER, 
    content= question
)
print(f"Created message: {message}")

Created message: {'id': 'msg_9J3zCpBEQ2gx1LmE5JbUNWtw', 'object': 'thread.message', 'created_at': 1751467171, 'assistant_id': None, 'thread_id': 'thread_M7JWwNMqn9JB5Eer2TVjI6P3', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Quali sono i modelli Azure OpenAI in scadenza e con cosa posso sostituirli? Fornisci dettagli per ogni modello e versione, evidenziando le differenze tra le versioni. Includi anche le date di scadenza e le versioni alternative consigliate.', 'annotations': []}}], 'attachments': [], 'metadata': {}}


In [10]:
run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)

In [13]:
run =  project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
run

{'id': 'run_Xprmz5JfihjrcRwRcnYZqGWT', 'object': 'thread.run', 'created_at': 1751467221, 'assistant_id': 'asst_63fy6vYue8aNd6OEBFQcwL1W', 'thread_id': 'thread_M7JWwNMqn9JB5Eer2TVjI6P3', 'status': 'requires_action', 'started_at': 1751467221, 'expires_at': 1751503221, 'cancelled_at': None, 'failed_at': None, 'completed_at': None, 'required_action': {'type': 'submit_tool_approval', 'submit_tool_approval': {'tool_calls': [{'id': 'call_90ouH8Uw62XcDppIte9bYynH', 'type': 'mcp', 'arguments': '{"question":"Quali sono i modelli Azure OpenAI in scadenza e con cosa posso sostituirli? Fornisci dettagli per ogni modello e versione, evidenziando le differenze tra le versioni. Includi anche le date di scadenza e le versioni alternative consigliate."}', 'name': 'microsoft_docs_search', 'server_label': 'microsoft_docs_search'}]}}, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are a customer support chatbot. Use the tools provided and your knowledge base to best respond to customer queries

In [15]:
import time
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")

if run.status == "failed":
    print(f"Run error: {run.last_error}")

run_steps = project_client.agents.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Run step: {step.id}, status: {step.status}, type: {step.type}")
    if step.type == "tool_calls":
        print(f"Tool call details:")
        for tool_call in step.step_details.tool_calls:
            print(json.dumps(tool_call.as_dict(), indent=2))

messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for data_point in messages:
    last_message_content = data_point.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"{data_point.role}: {last_message_content.text.value}")

Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION
Run status: RunStatus.REQUIRES_ACTION


KeyboardInterrupt: 

# Run the agent syncrhonously

In [ ]:
%%time

# Run the agent
run = project_client.agents.runs.create_and_process\
    (thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}.\n\nRun: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

# Fetch messages from the thread after the agent run execution

In [ ]:
from azure.ai.agents.models import MessageTextContent, MessageImageFileContent, MessageTextFileCitationAnnotation, MessageTextFilePathAnnotation

image_files = []
annotations = []
citations = []

if run.status == 'completed':
    messages = project_client.agents.messages.list(thread_id=thread.id)
    messages_list = list(project_client.agents.messages.list(thread_id=thread.id))  # Convert iterator to a list
    messages_nr = len(messages_list)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages_list), 1):
        j = 0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j +=1
            if (type(c) is MessageTextContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageTextContent) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    if type(a) is MessageTextFileCitationAnnotation:
                        print(f">>> Citation in MessageTextContent {j} of message {i}: {a}\n")
                        citations.append(a)
                    elif type(a) is MessageTextFilePathAnnotation:
                        print(f">>> Annotation in MessageTextContent {j} of message {i}: {a}\n")
                        annotations.append(a)
            elif (type(c) is MessageImageFileContent):
                print(f"\nMessage {i} / CONTENT {j} (MessageImageFileContent) --> image_file id: {c.image_file.file_id}")
                image_files.append(c.image_file.file_id)

else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

# Print annotations from the messages

In [ ]:
print (f"Nr. of file path annotations: {len(annotations)}\n")

i=0
for a in annotations:
    i += 1
    print(f"{i} - File annotation paths: {a.text}")
    file_name = a.text.split('/')[-1]
    file_id = a.file_path.file_id

    #agents_client.files.save(file_id=file_id, file_name=file_name)
    project_client.agents.files.save(file_id=file_id, file_name=file_name)
    print(f"\n>>> file <{file_id}> saved as <{file_name}>")
    
    # project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name)
    print(f"File annotation {i} saved as file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

# Fetch citations from the messages

In [ ]:
print (f"Nr. of file path citations: {len(citations)}\n")

i=0
for a in citations:
    i += 1
    print(f"{i} - citation: {a}")

# Retrieve and download eventual images

In [ ]:
print (f"Nr. of image contents: {len(image_files)}\n")

i=0
# Generate an image file for the bar chart
for image_file in image_files:
    i += 1
    print(f"{i} - Image file id: {image_file}")
    file_name = f"{image_file}.png"
    project_client.agents.files.save(file_id=image_file, file_name=file_name)
    print(f"Image content {i} file to: {os.getcwd()}/{file_name}")
    image = mpimg.imread(f"{os.getcwd()}/{file_name}") # read the image
    plt.imshow(image)
    plt.show()

# Collect all resources for this project

In [ ]:
all_agents = list_all_agents(client=project_client)
print(all_agents["summary"])

all_threads = list_all_threads(client=project_client)
print(all_threads["summary"])

all_files = list_all_files(client=project_client)
print(all_files["summary"])

all_runs = list_all_runs(client=project_client)
print(all_runs["summary"])

# all_runsteps=list_all_runsteps(project_client)
# print(all_runsteps["summary"])

# all_messages = list_all_messages(project_client)
# print(all_messages["summary"])

all_vectorstores = list_all_vectorstores(client=project_client)
print(all_vectorstores["summary"])

# Teardown for all resources

In [ ]:
# delete all vector stores

i=0
for vector_store in all_vectorstores["content"]:
    i += 1
    project_client.agents.vector_stores.delete(vector_store_id=vector_store.id)
    print(f"{i} - Vector store <{vector_store.id}> has been deleted")

all_vectorstores = list_all_vectorstores(client=project_client)

print(f"Vector stores deleted: {i}\n")

In [ ]:
# delete all files

i=0
for file in all_files['content']:
    i += 1
    project_client.agents.files.delete(file_id=file.id)
    print(f"{i} - File <{file.filename}> ({file.id}) has been deleted")

all_files = list_all_files(project_client)

print(f"Files deleted: {i}\n")

In [ ]:
# delete all threads

i=0
for thread in all_threads["content"]:
    i += 1
    project_client.agents.threads.delete(thread_id=thread.id)
    print(f"{i} - Thread <{thread.id}> has been deleted")

all_threads = list_all_threads(project_client)

print(f"Threads deleted: {i}\n")

In [ ]:
# delete all agents

i=0
for agent in all_agents["content"]:
    i += 1
    project_client.agents.delete_agent(agent_id=agent.id)
    print(f"{i} - Agent <{agent.id}> has been deleted")

all_agents = list_all_agents(client=project_client)

print(f"Agents deleted: {i}\n")

# HIC SUNT LEONES